In [1]:
import os
os.chdir("../")

In [10]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    metric_file_name : Path
    target_column : str

In [11]:
from phishing.constants import *
from phishing.utils.common import read_yaml,create_directories

In [12]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config=self.config.model_evaluation
        schema=self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config=ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,


        )
        return model_evaluation_config


In [13]:
import os 
import pandas as pd 
from sklearn.metrics import accuracy_score,confusion_matrix
import numpy as  np 
import joblib 
from phishing.utils.common import save_json,load_json


In [16]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self,actual, pred):
        acc_score = accuracy_score(actual, pred)
        confusion = confusion_matrix(actual, pred)
        return acc_score, confusion
    
    def evaluate(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]



        predicted_qualities = model.predict(test_x)

        (acc_score, confusion) = self.eval_metrics(test_y, predicted_qualities)
            
            # Saving metrics as local
        scores = {"accuracy":acc_score }
        save_json(path=Path(self.config.metric_file_name), data=scores)

        return scores



        


In [17]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluate = ModelEvaluation(model_evaluation_config)
    model_evaluate.evaluate()
except Exception as e:
    raise e